In [3]:
import re
import math
import time
import requests
import datetime
import urllib.parse

from datetime import date
from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from tqdm.auto import tqdm

## I. Tests for analyse webservice

TODO: Find corresponding captions and extend tests

In [1]:
def analyze_text(text: str):
    """
    Uses morphological analyser to find lemmas of space separated words.
    Returns a dataframe with columns wordform, lemma and sublemmas.
    """

    # Webservice call
    analyzer_query = "https://smart-search.tartunlp.ai/api/analyser/process"
    headers = {"Content-Type": "application/json; charset=utf-8"}
    post_data_template = {'params': {"vmetajson": ["--guess"]}, 'content': text}
    response = requests.post(analyzer_query, json=post_data_template, headers=headers)
    assert response.ok, "Webservice failed"
    response = response.json()

    # Count the number of analyses over all tokens
    analysis_count = sum(map(lambda x: len(x['features']['mrf']), response['annotations']['tokens']))
    tbl = DataFrame({
        'index': [None] * analysis_count,
        'wordform': [None] * analysis_count,
        'lemma': [None] * analysis_count,
        'sublemmas': [None] * analysis_count})

    # Unwind list of list into table
    row = 0
    for idx, token in enumerate(response['annotations']['tokens']):
        for annotation in token['features']['mrf']:
            tbl.loc[row, 'index'] = idx
            tbl.loc[row, 'wordform'] = token['features']['token']
            tbl.loc[row, 'lemma'] = annotation['lisamärkideta']
            tbl.loc[row, 'sublemmas'] = tuple(annotation['komponendid'])
            row += 1

    tbl = tbl.drop_duplicates().reset_index(drop=True)
    tbl['sublemmas'] = tbl['sublemmas'].map(list)

    # Assign lemma if there is no sublemmas for uniformity
    idx = tbl['sublemmas'].map(len) == 0
    tbl.loc[idx, 'sublemmas'] = tbl.loc[idx, 'lemma'].map(lambda x: [x])
    return tbl

In [5]:
display(analyze_text('„Talleks”'))
display(analyze_text('"Balti'))
display(analyze_text('“Lepingu'))
display(analyze_text('(EAEC)'))
display(analyze_text('Ühendust,'))
display(analyze_text('«Õigusabi'))

,index,wordform,lemma,sublemmas
0,0,„Talleks”,tall,[tall]


,index,wordform,lemma,sublemmas
0,0,"""Balti",balti,[balti]


,index,wordform,lemma,sublemmas
0,0,“Lepingu,leping,[leping]


,index,wordform,lemma,sublemmas
0,0,(EAEC),EAEC,[EAEC]


,index,wordform,lemma,sublemmas
0,0,"Ühendust,",ühendus,[ühendus]


,index,wordform,lemma,sublemmas
0,0,«Õigusabi,õigusabi,"[õigus, abi]"


### I.A Initial analysis of all captions

* We need to analyse all the document captions and get occurence counts for wordforms. 
* In this analysis we ignore the fact that some wordforms have multiple lemmas and treat each row with the same weight.

In [8]:
sources = read_csv('../../results/source_texts/state_laws.csv', header=0)

# result = [None] *  len(sources)
# for i, caption in tqdm(enumerate(sources['document_title']), total=len(sources)):
#    result[i] = analyze_document_caption(caption).assign(doc_id = i)

# result = concat(result, axis=0)
# display(result)

### Huvitavad sõnavormid

* Strasbourg'i
* International'i
* A/S
* U.K
* P.L.C

TODO: Otsida vastavad pealkirjad ülesse ja kontrollida tulemust.